<a href="https://colab.research.google.com/github/weathon/3d2smile/blob/main/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget file.weasoft.com/images_rows.csv

In [ ]:
!wget file.weasoft.com/29_4010.pt

In [ ]:
!wget http://file.weasoft.com/swin_transform_focalloss.pth -nv

2024-01-20 00:36:02 URL:http://file.weasoft.com/swin_transform_focalloss.pth [2555716718/2555716718] -> "swin_transform_focalloss.pth" [1]


In [ ]:
!git clone https://github.com/suanfaxiaohuo/SwinOCSR.git
!pip install focal_loss_torch wandb
!pip3 install deepsmiles yacs tqdm

In [ ]:
import base64
import pandas as pd
import os
df = pd.read_csv("images_rows.csv")
os.makedirs("images",exist_ok=1)
def save_img(id):
  row = df[df["id"]==id]
  print(row["image"])
  with open(f"images/{id}.png", "wb") as fh:
      fh.write(base64.decodebytes(bytes(row["image"].item().replace("data:image/png;base64,",""), "utf-8")))
df["id"].apply(save_img)

In [ ]:
import sys
sys.path.append("./SwinOCSR/model/Swin-transformer-focalloss")
sys.path.append("./SwinOCSR/model/")

In [58]:
import sys
import os
import argparse
from tqdm import tqdm
import deepsmiles
from typing import Any, cast, Callable, List, Tuple, Union
from PIL import Image

import torch
import torchvision
import torchvision.transforms as transforms

import numpy as np

In [45]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [47]:
mod = torch.load("29_4010.pt", map_location=device) #it is model not dict

In [52]:
def pad_pack(sequences):
    maxlen = max(map(len, sequences))
    batch = torch.LongTensor(len(sequences),maxlen).fill_(0)
    for i,x in enumerate(sequences):
        batch[i,:len(x)] = torch.LongTensor(x)
    return batch, maxlen

In [53]:
class SMILESGenerator(torch.nn.Module):
  def __init__(self, encoder, decoder):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder

  def forward(self, image, text_in_, max_len, beam): #just changed beam=1 and it runs so beam cannot be random number???  yeah it has to been factor of 676, which is 2 2 13 13 but original paper did not use beam search
    image_feature = self.encoder(image)
    top_n = list([list(i) for i in torch.tensor(text_in_).repeat(beam,1).detach().cpu().numpy()])
    # print(top_n)
    image_feature = image_feature.repeat_interleave(beam, dim=0)#.repeat_interleave(beam)
    for i in range(max_len):
      if beam == 1:
        padded_text, l = pad_pack(top_n)
        padded_text = padded_text.to(device)
        out = self.decoder(padded_text, image_feature, x_mask=triangle_mask(l).to(device))
        out = out[0,-1,:]
        next = torch.argmax(out)
        top_n[0] += [next.detach().cpu().item()]
        if next == 78:
          return top_n
        continue
      padded_text, l = pad_pack(top_n)
      padded_text = padded_text.to(device)
      out = self.decoder(padded_text, image_feature, x_mask=triangle_mask(l).to(device))
      out = out[:,-1,:]
      next = torch.topk(torch.flatten(out), beam)
      indices2d = np.array(np.unravel_index(next.indices.cpu().numpy(), out.shape)).T
      new_top_n = []
      count = 0
      for j in indices2d:
        beam_number = j[0]
        char = j[1]
        if char == 78:
          count += 1
          new_top_n.append(top_n[beam_number])
        else:
          new_top_n.append(top_n[beam_number] + [char])
      if count == beam:
        return new_top_n
      top_n = new_top_n
    return top_n

In [55]:
gen = SMILESGenerator(mod.encoder, mod.decoder)

In [60]:
def triangle_mask(size):
    mask = 1- np.triu(np.ones((1, size, size)),k=1).astype('uint8')
    mask = torch.autograd.Variable(torch.from_numpy(mask))
    return mask

In [62]:
reversed_word_map = {}
for i in transformer_.word_map.keys():
 reversed_word_map[transformer_.word_map[i]] = i

In [84]:
test = torch.tensor(np.array(Image.open("Untitled.png").resize((400,400)))).unsqueeze(0).permute(0, 3, 1, 2).to(torch.float32)

In [85]:
"".join([reversed_word_map[i] for i in gen(test, [[77]], 500, 1)[0]])

'<start>[S-].C=S)[S+].[Cl-]<end>'